In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from operator import add
import nltk
import json
from time import time
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("TestApp")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.shuffle.service.enabled", False)\
        .getOrCreate()

#        .config("spark.executor.cores",2)\
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

In [3]:
"""rdd2 = spark_context.newAPIHadoopFile(
    'hdfs://localhost:9000/user/ubuntu/RC_2006-02',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text'
)
"""
rdd = spark_context.textFile('file://///home/ubuntu/RC_2007-12')

In [4]:
rdd.take(3)

['{"author":"[deleted]","id":"c02khd5","subreddit":"politics","author_flair_css_class":null,"downs":0,"body":"The fact remains that Chavez does not understand economics. He\'s imposed [price controls](http://africa.reuters.com/odd/news/usnN30300735.html) and now even toilet paper is short. The government blames hoarding -- yeah, that\'s what happens when you impose price controls.\\n\\nThe better way to help poor people is to **give them money**. If you\'re afraid they\'ll waste their money on drugs, give them **vouchers**. Further, Chavez [heavily subsidizes oil](http://www.nytimes.com/2007/10/30/world/americas/30venezuela.html?_r=1&amp;oref=slogin), which is actually costing enormous amounts of money.\\n\\nChavez will never make any progress while he continues to work against economics.\\n\\nThe \\"ruling elites\\" don\'t rule the country anymore. I\'d be more worried about who is holding Chavez and his guys accountable!\\n\\nBy the way, I\'m a hardcore liberal, but I understand that

In [5]:
_tstart_stack = []
def tic():
    _tstart_stack.append(time())
def toc(fmt="Elapsed: %s s"):
    print (fmt % (time() - _tstart_stack.pop()))

In [6]:
gram_groups = dict()
gram_groups['Adjectives'] = ['JJ', 'JJR', 'JJS']
gram_groups['Nouns'] = ['NN', 'NNS', 'NNP', 'NNPS']
gram_groups['Pronouns'] = ['PRP', 'PRP$']
gram_groups['Adverbs'] = ['RB', 'RBR', 'RBS']
gram_groups['Verb'] = ['VB', 'VBG', 'VBD', 'VBN', 'VBP', 'VBZ']

def get_split_body(rdd):
    body = rdd\
    .map(lambda line: json.loads(line)['body'].strip().split())\
    
    return body

def check_gram_grp(tag_tuple):
    word, tag = tag_tuple
    for supergroup, subgroups in gram_groups.items():
                if tag in subgroups:
                    return supergroup
    return None

def categorize_words(split_rdd):
    cat_words = split_rdd.flatMap(lambda word: nltk.pos_tag(word))\
    .map(lambda tupl: (check_gram_grp(tupl), 1))\
    .filter(lambda x: x[0] != None)
   
    return cat_words

def final_test(rdd):
    split = get_split_body(rdd)
    categorized = categorize_words(split)
    group_counts = categorized.reduceByKey(add).collect()
    return group_counts

In [7]:

tic()
final_test(rdd)
toc()

Elapsed: 359.29846382141113 s


In [8]:
rdd_2_3 = rdd.sample(False, 2/3)

In [9]:
tic()
final_test(rdd_2_3)
toc()

Elapsed: 241.35568690299988 s


In [46]:
rdd.count()

363390